DELABEL / Remove white parts like MLO or CC signs 
(beyaz kısımları kaldırır)

In [ ]:
import time
import cv2
import numpy as np

def delabel(pixels):
    bin_pixels = cv2.threshold(pixels, 10, 255, cv2.THRESH_BINARY)[1]
    contours, _ = cv2.findContours(bin_pixels, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contour = max(contours, key=cv2.contourArea)
    mask = np.zeros(pixels.shape, np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, cv2.FILLED)
    out = cv2.bitwise_and(pixels,mask)
    
    return out

img = cv2.imread('Images/RCC.png',0)
cv2.imwrite('delabel.png',delabel(img))

RESIZE THE IMAGE

In [ ]:
def resize(image):

    image = cv2.resize(image, (640,640), interpolation = cv2.INTER_AREA)
    return image

CLAHE

In [ ]:
import cv2
import numpy as np

#tile_grid_size=(8,8)):
def clahe(img, clip_limit=2.0, tile_grid_size=(8,8)):
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    img_clahe = clahe.apply(img)
    return img_clahe

img = cv2.imread('Delabel.png',0)

cv2.imwrite('clahe.png',clahe(img))

BREAST SIDE

In [ ]:
import cv2
import numpy as np

def b_type(img):
    w = img.shape[1]
    right_brightness = np.sum(img[:, w-20: w])
    left_brightness = np.sum(img[:, 0: 20])

    if right_brightness > left_brightness:
        return "Right"
    return "Left"


BRIGHT SIDE AND ROI

In [ ]:
import cv2
import numpy as np
import time

def crop(image):

    gray = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)

    ret,thresh = cv2.threshold(gray,10,255,cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea)
    x,y,w,h = cv2.boundingRect(largest_contour)
    cropped = image[y:y+h, x:x+w]
    return cropped

#Pectoral kas alınır ve geri kalan önemli bölge elde edilir.
#Removing pectoral muscle
def cropRoi(img): 
    end = -1
    w = img.shape[1]
    h = img.shape[0]
    
    right_brightness = np.sum(img[:, w-20: w])
    left_brightness = np.sum(img[:, 0: 20])

    if right_brightness > left_brightness:
        
        for i in range(1023, -1, -1):
            if np.all(img[1411][i] < 10):
                end = i
                break
                
        myROI = [(0,0),(int(h*0.55),0),(h,int(h*0.8)),(int(w*0.8),100),(0,0)]
        h = img.shape[0]
        return crop(cv2.fillPoly(img, [np.array(myROI)], 0))
        
    
    end = -1
    for i in range(int(h/3)):
        if np.all(img[10][i] < 10):
    	    end = i
    	    break
    myROI = [(0,int(w*0.10)),(0,int(h*0.55)),(int(w*0.10),int(h*0.20)),(int(w*0.50),0),(w,0)]
    
    return crop(cv2.fillPoly(img, [np.array(myROI)], 0))
   
#out = cv2.imread('img1.png',0)
#cv2.imwrite('img2.png',crop(out))
#cv2.imwrite('img3.png',cropRoi(crop(out)))

MLO CC Separator

In [ ]:
import cv2
import numpy as np

def detect_mammogram_view(image):
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for cnt in contours:
        (x, y, w, h) = cv2.boundingRect(cnt)
        aspect_ratio = float(w) / h
        return "MLO" if aspect_ratio > 0.8 else "CC"